# Solution Planning

## Business Problem

1 - Selecionar os clientes mais valiosos para integrar o programa de fidelidade "Loyals".

2 - Responder a questões de negócio ao time de marketing.

### Output

O que vou entregar? / Onde o time de negócio quer ver?

1 - Lista em xls / enviar por email. Deve conter clientes que irão aderir ao Loyals (programa de fidelidade)

Formato: 
```client_id | is_loyal
     123  |  yes
     124  |  no ```
                     
2 - Relatório em pdf / enviar por email e apresentar ao time de marketing.

O relatório deve responder as seguintes questões de negócio:
- 1 - Quem são as pessoas elegíveis para participar do programa Loyals?
- 2 - Quantos clientes farão parte do grupo?
- 3 - Quais são as principais características desses clientes?
- 4 - Qual a porcentagem de contribuição de faturamento, vinda do Loyals?
- 5 - Qual a expectativa de faturamento desse grupo para os próximos meses?
- 6 - Quais as condições para uma pessoa ser elegível ao Loyals? 
- 7 - Quais as condições para uma pessoa ser removida do Loyals?
- 8 - Qual a garantia que o programa Loyals é melhor que o restante da base?
- 9 - Quais ações do time de marketing pode realizar para aumentar o faturamento?

### Input

Fontes de dados:
    Dataset "Ecommerce.csv", contendo as vendas de e-commerce do período de um ano.

Ferramentas:
    Python 3.8.12, Jupyter Notebook, Git, Github.

### Processo

1. **Quem são as pessoas elegíveis para participar do programa de Insiders ?**
    - O que é ser elegível ? O que é um cliente "valioso" para a empresa ?
        
        - Faturamento:

**Tipo de problema:**
Separação de clientes por grupos.
    
**Principais métodos:**
    Clusterização.

**Perguntas de negócio:**

1. **Quem são as pessoas elegíveis para participar do programa Loyals?**
    - O que é ser elegível? / O que são clientes de maior "valor"? (de acordo com área de negócio)

Faturamento:
 - Alto ticket médio
 - Alto LTV (soma da receita do cliente conosco)
 - Baixa recência (tempo desde a última compra)
 - Alto basket size (qtd produtos comprados por compra)
 - Baixa probabilidade de churn (usaria a saída de um modelo)
 - Alta Previsão LTV (usaria saída de um modelo)
 - Alta propensão de compra (usaria a saída de um modelo)
 
Custo:
 - Baixa taxa de devolução
 
Experiência de compra:
 - Média alta das avaliações
 
PS: na prática, acima será o feature engeneering, pois precisarei criar estas features.   

2 - Qual a porcentagem de clientes do Loyals em relação ao todo?
- Numero total de clientes / número de Loyals 
    
3 - Quais são as principais características desses clientes? (para look alike: prospectar clientes parecidos na internet)

Escrever caracterísiticas do cliente:
- Idade
- Localização
Escrever caracterísitca do consumo:
- Features da clusterização (questão 1)
    
4 - Qual a porcentagem de contribuição de faturamento, vinda do Loyals?
- Faturamento total do ano / faturamento do Loyals

5 - Qual a expectativa de faturamento desse grupo para os próximos meses? (isso tem meta)
- LTV do Loyals (com média móvel, time series, arima..)
- Análise de Cohort (com tempo, localização, produto..)
       
6 - Quais as condições para uma pessoa ser elegível ao Loyals? (threshold de preço? Várias vars?) 
- Definir a periodicidade que vai rodar (a cada 1 mês, 3 meses..) 
- A pessoa precisa ser similar com outra pessoa do grupo Loyals

7 - Quais as condições para uma pessoa ser removida do Loyals?
- A pessoa precisa ser diferente de outra pessoa do grupo Loyals
    
8 - Qual a garantia que o programa Loyals é melhor que o restante da base?
- Teste A/B
- Teste A/B Bayesiano
- Teste de hipótese

9 - Quais ações do time de marketing pode realizar para aumentar o faturamento? 
- Desconto
- Preferência de compra
- Frete
- Visita a emrpesa
- Oferecer personal stylist
- Recomendar cross sell 
- Oferecer conteúdo expclusivo

## Benchmarking de soluções

### Desk Research

Leitura de artigos sobre customer segmentation na internet, para compreender o que o mercado está fazendo.

Identificar a partir das soluções do mercado, com o time de negócio, o que podemos fazer como MVP.

# Environment Preparation

## Imports

In [1]:
#plot and handle
import pandas            as pd
import matplotlib.pyplot as plt

#display
import seaborn           as sns
from IPython.core.display     import HTML
#from IPython.display          import Image

## Helper Functions

In [2]:
def jupyter_settings():
    """ Optimize general settings, standardize plot sizes, etc. """
    %matplotlib inline
    plt.style.use( 'bmh' )
    plt.rcParams['figure.figsize'] = [12, 6]
    plt.rcParams['font.size'] = 20
    display( HTML( '<style>.container { width:100% !important; }</style>') )
    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    pd.set_option( 'display.expand_frame_repr', False )
    sns.set()
    pd.set_option('display.max_columns', 30)
    pd.set_option('display.max_rows', 30)
jupyter_settings()